1. Web scrapping Wikipedia page for list of postal codes of Canada and stroing the data in a Pandas dataframe

In [1]:
#wiki page link 
link = r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 

In [3]:
canada_postcode_wiki = requests.get(link).text
soup = BeautifulSoup(canada_postcode_wiki, 'html5lib')

In [4]:
#Explore Each cell in the table and break down the contents into Postal Code , Borough and Neighborhood
table=soup.find('table')
for row in table.findAll('td'):
    if row.span.text=='Not assigned':
        pass
    else:
        print(row.span.text)
        print(row.p.text)
        print(row.span.text.split("(")[0])
        print(row.span.text.split("(")[1].replace(")",' ').replace('/',',').strip(' '))
        break

North York(Parkwoods)
M3ANorth York(Parkwoods)

North York
Parkwoods


Webscarapping 
Beautiful Soup Code to parse and extract the data from the wiki table and convert it into a pandas data frame

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        #cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        cell['Neighborhood'] = row.span.text.split("(")[1].replace(")",' ').replace('/',',').strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


Extracted dataframe shape

In [6]:
df.shape

(103, 3)

2. Adding Latitude and logitude data for each Postal Code (Using the geospatial csv file)

In [7]:
import geocoder

In [8]:
geospatical_file = r"D:\UserData\z003ydzv\OneDrive - Siemens AG\D Drive\Data Science Certification - IBM\10. Applied Data Science Capstone\Coursera_Capstone\Geospatial_Coordinates.csv"

geospatical_df=pd.read_csv(geospatical_file)
geospatical_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
#add blank columns for latitude and longitude in main dataframe 
df['Latitude']=''
df['Longitude']=''

#look up geospatcial dataframe and add latitude and longitude into main dataframe 

for i, rowi in df.iterrows():
    for j, rowj in geospatical_df.iterrows():
        if rowj["Postal Code"]== rowi["PostalCode"]:
            rowi["Latitude"]=rowj["Latitude"]
            rowi["Longitude"]=rowj["Longitude"]

df
    

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.66586,-79.38316
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509


3. Geospatical clustering analysis

Importing the required Libraries

In [10]:
import numpy as np # library to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Co oordinates of Toronto Canada

In [11]:
address = "Toronto, CA"

geolocator = Nominatim(user_agent="ny_explorer")
location_toronto = geolocator.geocode(address)
latitude_toronto = location_toronto.latitude
longitude_toronto = location_toronto.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Map of Toronto with the neighbourhoods superimposed on top of the same

In [12]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare API credentials

In [13]:
CLIENT_ID = 'QWN5GN1ZVLHMKCCIBHR3QG54YUICXZIM03HYE0UGYKSCX5HB' # your Foursquare ID
CLIENT_SECRET = 'XWNW5XRMNR2ERHEKUC0VIQD5KRCGNDIYSQUGI1FSCAYOYWZ4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QWN5GN1ZVLHMKCCIBHR3QG54YUICXZIM03HYE0UGYKSCX5HB
CLIENT_SECRET:XWNW5XRMNR2ERHEKUC0VIQD5KRCGNDIYSQUGI1FSCAYOYWZ4


Explore the venues of all the neighbourhoods in Toronto

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills North
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
The Danforth  East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village

In [16]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Neighbourhood Analysis

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#adding neighbourhood name back to the data frame 
toronto_onehot['Neighborhood']= toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping the data frame 

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood , Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park , Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale South,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.028571,0.0,0.0,0.0,0.0,0.0
96,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.142857,0.000000,0.0,0.0,0.0,0.0,0.0


Create a dataframe with the top 10 most common features

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Yoga Studio,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
1,"Alderwood , Long Branch",Pizza Place,Gym,Sandwich Place,Athletics & Sports,Pub,Coffee Shop,Playground,Pharmacy,Park,Other Great Outdoors
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Pharmacy,Shopping Mall,Middle Eastern Restaurant,Mobile Phone Shop,Sandwich Place,Fried Chicken Joint,Frozen Yogurt Shop,Supermarket
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Yoga Studio
4,"Bedford Park , Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Pet Store,Toy / Game Store,Sushi Restaurant,Restaurant,Juice Bar,Liquor Store,Fast Food Restaurant


K Means Clustering 

In [21]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods_venues_sorted

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(df.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Yoga Studio,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,M1S,Scarborough,43.7942,-79.262029
1,0,"Alderwood , Long Branch",Pizza Place,Gym,Sandwich Place,Athletics & Sports,Pub,Coffee Shop,Playground,Pharmacy,Park,Other Great Outdoors,M8W,Etobicoke,43.602414,-79.543484
2,0,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Pharmacy,Shopping Mall,Middle Eastern Restaurant,Mobile Phone Shop,Sandwich Place,Fried Chicken Joint,Frozen Yogurt Shop,Supermarket,M3H,North York,43.754328,-79.442259
3,0,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Yoga Studio,M2K,North York,43.786947,-79.385975
4,0,"Bedford Park , Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Pet Store,Toy / Game Store,Sushi Restaurant,Restaurant,Juice Bar,Liquor Store,Fast Food Restaurant,M5M,North York,43.733283,-79.41975


Plotting of clusters 

In [25]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster Examination 

Cluster 1

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Yoga Studio,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,M1S,Scarborough,43.7942,-79.262029
1,0,"Alderwood , Long Branch",Pizza Place,Gym,Sandwich Place,Athletics & Sports,Pub,Coffee Shop,Playground,Pharmacy,Park,Other Great Outdoors,M8W,Etobicoke,43.602414,-79.543484
2,0,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Pharmacy,Shopping Mall,Middle Eastern Restaurant,Mobile Phone Shop,Sandwich Place,Fried Chicken Joint,Frozen Yogurt Shop,Supermarket,M3H,North York,43.754328,-79.442259
3,0,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Yoga Studio,M2K,North York,43.786947,-79.385975
4,0,"Bedford Park , Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Pet Store,Toy / Game Store,Sushi Restaurant,Restaurant,Juice Bar,Liquor Store,Fast Food Restaurant,M5M,North York,43.733283,-79.41975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0,"Wexford , Maryvale",Middle Eastern Restaurant,Auto Garage,Bakery,Sandwich Place,Vietnamese Restaurant,Smoke Shop,Shopping Mall,Miscellaneous Shop,Molecular Gastronomy Restaurant,Modern European Restaurant,M1R,Scarborough,43.750071,-79.295849
95,0,Willowdale South,Coffee Shop,Ramen Restaurant,Japanese Restaurant,Sushi Restaurant,Pizza Place,Shopping Mall,Café,Restaurant,Discount Store,Fast Food Restaurant,M2N,North York,43.77012,-79.408493
96,0,Willowdale West,Pizza Place,Pharmacy,Coffee Shop,Grocery Store,Discount Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,M2R,North York,43.782736,-79.442259
97,0,Woburn,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,M1G,Scarborough,43.770992,-79.216917


Cluster 1 has places with a lot of venues for cofee shops, banks, cafes etc

Cluster 2

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
9,1,Caledonia-Fairbanks,Park,Women's Store,Bar,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant,M6E,York,43.689026,-79.453512
35,1,Forest Hill North & West,Jewelry Store,Park,Trail,Sushi Restaurant,Yoga Studio,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,M5P,Central Toronto,43.696948,-79.411307
49,1,"Kingsview Village , St. Phillips , Martin Grov...",Sandwich Place,Park,Mobile Phone Shop,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Yoga Studio,M9R,Etobicoke,43.688905,-79.554724
51,1,Lawrence Park,Photography Studio,Bus Line,Park,Swim School,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant,M4N,Central Toronto,43.72802,-79.38879
55,1,"Milliken , Agincourt North , Steeles East , L'...",Playground,Park,Intersection,Pharmacy,Pet Store,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,M1V,Scarborough,43.815252,-79.284577
57,1,"Moore Park , Summerhill East",Tennis Court,Lawyer,Park,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,M4T,Central Toronto,43.689574,-79.38316
59,1,"North Park , Maple Leaf Park , Upwood Park",Basketball Court,Bakery,Construction & Landscaping,Park,Yoga Studio,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,M6L,North York,43.713756,-79.490074
66,1,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,M3A,North York,43.753259,-79.329656
69,1,Rosedale,Park,Playground,Trail,Yoga Studio,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,M4W,Downtown Toronto,43.679563,-79.377529
83,1,The Danforth East,Convenience Store,Park,Yoga Studio,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,M4J,East YorkEast Toronto,43.685347,-79.338106


Cluster 2 has a lot of parks 

Cluster 3

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
44,2,"Humberlea , Emery",Baseball Field,Yoga Studio,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant,Movie Theater,M9M,North York,43.724766,-79.532242
62,2,"Old Mill South , King's Mill Park , Sunnylea ,...",Pool,Construction & Landscaping,Baseball Field,Yoga Studio,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,M8Y,Etobicoke,43.636258,-79.498509
85,2,"The Kingsway , Montgomery Road , Old Mill North",River,Pool,Yoga Studio,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant,M8X,Etobicoke,43.653654,-79.506944


Clsuter 3 has lot of pools, basketball fields, followed by yoga studio. 

Cluster 4

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3] 

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
25,3,Downsview East,Airport,Park,Yoga Studio,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,M3K,North York,43.737473,-79.464763
92,3,Weston,Park,Yoga Studio,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,M9N,York,43.706876,-79.518188
94,3,"Willowdale , Newtonbrook",Park,Yoga Studio,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,M2M,North York,43.789053,-79.408493


Cluster 4 has a lot of parks, yoga studios and middle eastern restaurants

Cluster 5

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
74,4,Scarborough Village,Playground,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant,Market,M1J,Scarborough,43.744734,-79.239476


Cluster 5 has playgrounds followed by miscellaneous shops and motels.